## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = range(1,1001)
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [8]:
# first
rdd.first()

1

In [11]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [12]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [14]:
# collect
print(rdd.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [15]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([2.48963710e-03, 1.15419715e+00, 5.06724962e-01, 2.49277357e+00,
       2.31289875e+00, 4.16906977e+00, 4.99360020e+00, 4.70964440e-01,
       6.71956535e+00, 7.55398768e+00, 1.06441706e+01, 4.58027218e+00,
       1.05027320e+01, 6.12938379e+00, 1.90570086e+00, 8.31666448e+00,
       3.67399103e+00, 9.00717726e+00, 4.98540195e+00, 1.43503707e+01,
       1.38464255e+01, 2.11924636e-01, 5.71761757e+00, 2.00740187e+01,
       2.45749579e+01, 1.69547140e+01, 2.37325600e+01, 4.37597151e+00,
       1.21329048e+01, 4.52048547e+00, 8.47704919e+00, 9.74512868e-01,
       2.06059080e+01, 4.82023263e+00, 1.81371422e+01, 1.33080603e+01,
       1.17057387e+01, 1.64693712e+01, 2.01142603e+00, 1.82644806e+01,
       1.92073265e+01, 3.72702213e+01, 6.93401706e+00, 2.86031071e+01,
       1.73270838e+01, 1.49824601e+01, 2.14116843e+01, 3.17964878e+00,
       3.46360834e+01, 4.22792143e+01, 2.62210227e+01, 2.68142242e+01,
       4.39928899e+01, 2.89767886e+01, 9.91962631e+00, 1.65545693e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [18]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(5)

[0.002489637103735398,
 1.1541971530071067,
 0.506724961944808,
 2.4927735679460765,
 2.312898746409636]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [19]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [20]:
# perform action to retrieve rdd values
revenue_minus_tax.take(5)

[0.0022904661354365666,
 1.0618613807665382,
 0.46618696498922335,
 2.2933516825103903,
 2.127866846696865]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [21]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [22]:
discounted.take(10)

[0.00206141952189291,
 0.9556752426898845,
 0.41956826849030104,
 2.064016514259351,
 1.9150801620271787,
 3.4519897686887893,
 4.134700969238142,
 0.38995855608956786,
 5.563800108357257,
 6.254701801806707]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [23]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[802.7823334730276,
 791.2318520666074,
 785.3349106821706,
 765.7150378921526,
 760.0545052206692,
 740.2678769151388,
 736.9584226903178,
 736.5613761008092,
 717.8774670742409,
 716.9340358997529,
 713.3826428007316,
 701.9209761128024,
 698.9920700915654,
 697.3669528940856,
 691.8112760592392]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [24]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[8] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [25]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.002489637103735398, 0.00206141952189291), (1.1541971530071067, 0.9556752426898845), (0.506724961944808, 0.41956826849030104), (2.4927735679460765, 2.064016514259351), (2.312898746409636, 1.9150801620271787), (4.1690697689478124, 3.4519897686887893), (4.993600204393891, 4.134700969238142), (0.4709644397217003, 0.38995855608956786), (6.719565348257556, 5.563800108357257), (7.5539876833414334, 6.254701801806707)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [27]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.002489637103735398, 0.00206141952189291, 1.1541971530071067, 0.9556752426898845, 0.506724961944808, 0.41956826849030104, 2.4927735679460765, 2.064016514259351, 2.312898746409636, 1.9150801620271787]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [30]:
# use the filter function
selected_items = price_items.filter(lambda x: x*.92*.9>300)

# calculate total remaining in inventory 
selected_items.count()

271

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [31]:
selected_items.reduce(lambda x,y :x + y)

151112.53079313552

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [44]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(9, 367.58723666763973),
 (29, 363.4307399576611),
 (46, 373.56505070987066),
 (19, 375.42964608453406),
 (7, 380.45749161362994),
 (37, 389.06967645274204),
 (21, 364.52863627491206)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [48]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y :x + y)
total_spent.take(10)

[(40, 2439.5205657148717),
 (20, 4048.6683534755684),
 (30, 3256.186581427667),
 (50, 3110.719497659677),
 (10, 3247.2853931147924),
 (41, 4543.098489511859),
 (1, 2664.337451925615),
 (31, 2611.669627180679),
 (11, 1833.3900536593965),
 (21, 1788.7088517124548)]

In [52]:
from operator import add
total_spent2 = sales_data.reduceByKey(add)
total_spent2.take(10)

[(10, 4528.642959031231),
 (30, 2291.0122053273108),
 (50, 3575.0423263041275),
 (40, 2506.003910920532),
 (20, 2722.617598459398),
 (41, 1930.4206024713062),
 (11, 3157.4331127913565),
 (1, 2260.534541395965),
 (21, 1897.5709575139786),
 (31, 4082.6756190000815)]

In [42]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(44, 5267.5962264791715),
 (30, 4857.52913906113),
 (18, 4680.581653755209),
 (7, 4635.6692718925515),
 (21, 4541.811282715404),
 (12, 4513.132689062008),
 (27, 4420.040443183558),
 (41, 4231.5390007186925),
 (26, 4221.303882811167),
 (23, 4190.477884400389),
 (5, 3881.7803148240346),
 (32, 3830.965280414954),
 (1, 3801.277743755023),
 (40, 3745.5700433498573),
 (19, 3512.795657992192),
 (36, 3408.254845693569),
 (13, 3388.7966442509933),
 (25, 3315.2111688433865),
 (20, 3310.75999529409),
 (48, 3282.426420862604),
 (38, 3189.2310284088685),
 (34, 3186.8487871863526),
 (9, 3149.8123733363054),
 (47, 3101.2357174785548),
 (43, 3006.201208076779),
 (10, 2990.6712097852187),
 (22, 2968.5296912926005),
 (28, 2847.005329953863),
 (15, 2835.3960610811187),
 (16, 2809.706157584843),
 (50, 2777.4624603339257),
 (4, 2767.236182918562),
 (2, 2650.853662295768),
 (8, 2595.8104495529374),
 (24, 2520.7300632665497),
 (17, 2289.8157461987944),
 (49, 2278.5263851596956),
 (11, 2204.311361205526),
 (

In [43]:
total_spent2.sortBy(lambda x: x[1],ascending = False).collect()

[(41, 6601.223194982086),
 (40, 6472.663877787461),
 (49, 5993.120283952966),
 (50, 5491.21166631734),
 (4, 5357.3059119157315),
 (21, 4826.828515414356),
 (16, 4628.976087857942),
 (8, 4487.91228844599),
 (38, 4404.399951094594),
 (48, 4031.5800251413834),
 (46, 4002.117811344517),
 (42, 3980.767000154594),
 (34, 3956.2384058848065),
 (26, 3865.1569793481235),
 (43, 3680.973315726877),
 (39, 3630.273731872536),
 (1, 3615.726268271739),
 (31, 3557.707872095605),
 (44, 3378.3600090347236),
 (17, 3351.3707027382056),
 (15, 3340.8561672634146),
 (13, 3272.873420804085),
 (10, 3209.3663934658803),
 (25, 3186.69876516613),
 (27, 3164.026371500009),
 (6, 3113.0874784863627),
 (2, 3099.0416594284115),
 (20, 2902.366479709592),
 (28, 2867.271912669668),
 (24, 2803.3335925136453),
 (14, 2669.4582632141382),
 (37, 2575.5317204671014),
 (3, 2526.543273255047),
 (32, 2501.102371653629),
 (11, 2159.666742548484),
 (7, 2131.898891308432),
 (19, 1882.2709237528506),
 (12, 1810.7709533630748),
 (5, 18

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [60]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(41, 10),
 (37, 10),
 (49, 9),
 (10, 9),
 (20, 9),
 (47, 8),
 (45, 8),
 (27, 8),
 (25, 8),
 (38, 8),
 (12, 8),
 (5, 8),
 (3, 7),
 (31, 7),
 (13, 7),
 (2, 7),
 (6, 7),
 (26, 7),
 (19, 6),
 (18, 6),
 (15, 6),
 (29, 6),
 (22, 6),
 (7, 5),
 (24, 5),
 (14, 5),
 (9, 5),
 (30, 5),
 (35, 5),
 (21, 4),
 (46, 4),
 (28, 4),
 (42, 4),
 (44, 4),
 (23, 4),
 (50, 4),
 (17, 3),
 (4, 3),
 (48, 3),
 (33, 3),
 (1, 3),
 (40, 3),
 (16, 3),
 (36, 3),
 (34, 3),
 (43, 3),
 (32, 3),
 (11, 2),
 (8, 2),
 (39, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
